# TP 2 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
DISPLAY_TREE = true
if DISPLAY_TREE
    # Les 2 lignes suivantes à décommenter sur les machines persos
    import Pkg; 
    Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
    using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound
end

LoadError: SystemError: opening file "/mnt/n7fs/ens/tp_cots/.julia/logs/manifest_usage.toml": Permission non accordée

### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [3]:
function testSondability_LP( capacity,  borne, BestProfit, Bestsol)
   
    TA, TO, TR = false, false, false
    
    

    if capacity < 0                                                                   # Test d'Admissibilité
        TA = true
        
        println("TA")
        
    elseif (borne < BestProfit)  # Test d'optimalite
        TO = true
        
        println("TO")
        
    elseif (capacity == 0) || (borne == BestProfit)                                   # Test de resolution
        TR = true
        
        println("TR")
       
        
    else
        
        println("non sondable")
        
    end
    return TA, TO, TR, Bestsol, BestProfit
end

testSondability_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [4]:
function separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left

    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end

    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible

    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop
end

exploreNextNode_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): 

In [5]:
function Bornes(price, weight, capacity, listobjs, listvals, BestProfit, Bestsol, option)
    
    borne = 0
    prix = 0
    max = 0

    if option == 1
        weightAct = 0
        priceAct = 0
        for i in 1:length(listobjs)
            if listvals[i] == 1
                if weightAct + weight[listobjs[i]] <= capacity
                    weightAct += weight[listobjs[i]]
                    priceAct += price[listobjs[i]]
                end
            end 
        end
    
        if priceAct >= BestProfit
            Bestsol = copy(listobjs)
            BestProfit = priceAct
        end
            
        if (length(listobjs) < length(price)) 
            for i in (length(listobjs)+1):length(price)
                r = price[i]/weight[i]
                if (!(i in listobjs)) 
                    if r > max        
                        max = r
                    end     
                end
            end
            borne = prix + max*capacity
        else
            borne = prix
        end
        
    elseif option == 2
        for i in 1:length(listobjs)
            if listvals[i] == 1
                capacity = capacity - weight[listobjs[i]]
                println("Capacity = ", capacity)
                prix = prix + price[listobjs[i]]
                
            end
        end
    
        deleteat!(price, listobjs)
        deleteat!(weight, listobjs)
    
        if capacity >= 0
            if prix > BestProfit
                BestProfit = prix
                Bestsol = copy(listobjs)
            end
        end
        
        
        # Ratio
        r = price./weight
        listR = [(r[i], weight[i]) for i in 1:length(price)]
        listR = sort(listR, by = x -> x[1], rev=true)
        prix1 = 0

        for i in 1:length(listR)
            if listR[i][2] <= capacity
                prix1 = prix1 + listR[i][1]*listR[i][2]
                capacity = capacity - listR[i][2]
            else
                break
            end
        end

        borne = prix + prix1
    end

return borne, capacity, BestProfit, Bestsol

end

Bornes (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [6]:
function solveKnapInstance(filename, option)

    price, weight, capacity = readKnaptxtInstance(filename)
    n = length(price)

    #model2, x = createModel_LP(price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=Int64[]
    listvals=Float64[]
    listnodes=Int64[]

    BestProfit::Float64=-1.0
    Bestsol=Float64[]

    current_node_number::Int64=0
    stop = false

    while(!stop)

        println("\nNode number ", current_node_number, " : \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        
        #create LP of current node
        #updateModele_LP!(model2, x, listobjs, listvals)
        
        #println(model2)
        
        print("Solve the LP model of the current node to compute its bound: start ...\n\n")

        #status = optimize!(model2)

        
        priceAct = deepcopy(price)
        weightAct = deepcopy(weight)
        capacityAct = capacity
        
        borne, capacityAc, Bestp, BestS = Bornes(priceAct, weightAct, capacityAct, listobjs,listvals, BestProfit, Bestsol, option)

        BestProfit = Bestp

        Bestsol = BestS



        TA, TO, TR = testSondability_LP( capacityAc,  borne, Bestp, BestS)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, length(price))
        else
            stop = exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
        end
        
        #resetAll_LP!(model2, x)

        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x = ", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


solveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [7]:
function solveNdisplayKnap(filename)

    println("\nBranch-and-Bound for solving a knapsack problem. \n\nSolving instance '" * filename * "'\n")

    # Option 1
    println("***** using option 1 *****")
    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = @time solveKnapInstance(filename, 1)
    
    # Affichage de l'arbre
    println("\n Branch-and-bound tree visualization : start display ...")
    display(graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree))
    println("... end display. \n\n")

    println("-----------------------------------------------\n")

    # Option 2
    println("***** using option 2 *****")
    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = @time solveKnapInstance(filename, 2)

    # Affichage de l'arbre
    println("\n Branch-and-bound tree visualization : start display ...")
    display(graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree))
    println("... end display. \n\n")

end

solveNdisplayKnap (generic function with 1 method)

In [8]:
INSTANCE = "test.opb.txt"
solveNdisplayKnap(INSTANCE)
println("press enter to exit ! ")



Branch-and-Bound for solving a knapsack problem. 

Solving instance 'test.opb.txt'

***** using option 1 *****

Node number 0 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

non sondable

branch on object 1


Node number 1 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

non sondable

branch on object 2


Node number 2 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

non sondable

branch on object 3


Node number 3 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

TO

Node number 4 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

TO

Node number 5 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

TO

Node number 6 : 
---------------

Solve the LP model of the current node to compute its bound: start ...

non sondable

branch on obje

LoadError: UndefVarError: graphplot not defined

### Questions préliminaires


### Résultats 

Questions préliminaires
- Q1. La règle de séparation utilisée est la règle lexicographique, qui consiste à sélectionner les objets dans l'ordre défini par leur position. Autrement dit, on choisit en priorité le premier objet non encore inclus dans la solution.

- Q2. La borne supérieure est obtenue en résolvant la relaxation linéaire du problème. Cela consiste à fixer les variables des objets déjà inclus à 111 et celles des objets exclus à 000.

- Q3.

Test d’Admissibilité (TA) : Une solution est admissible si la relaxation linéaire du problème fournit une solution faisable.

Test d’Optimalité (TO) : Un nœud est abandonné si la borne supérieure calculée pour ce nœud est inférieure au profit maximal déjà trouvé.

Test de Relaxation (TR) : Ce test est validé si la solution obtenue par la relaxation linéaire est entière.

- Q4. La stratégie d’exploration adoptée est celle de recherche en profondeur, aussi appelée Depth-First Search (DFS).





Code et Analyse
- Q1/Q2. Les calculs des bornes supérieures sont implémentés dans la fonction Bornes.

- Q3.

Règle de Séparation : L’algorithme conserve l’ordre lexicographique pour choisir les objets à brancher.

Test d’Admissibilité (TA) : Ce test réussit si la capacité restante du sac devient négative, ce qui indique que le sac dépasse sa capacité.

Test d’Optimalité (TO) : Le test est validé si la borne supérieure calculée pour un nœud est inférieure à la meilleure solution trouvée jusqu’à présent.

Test de Relaxation (TR) : Ce test est validé si une solution évidente est trouvée, comme lorsque la capacité restante est nulle ou si la solution actuelle est identique à une solution déjà identifiée.

Stratégie d’Exploration : L’algorithme utilise la stratégie de recherche en profondeur (DFS).

- Q4.

Calcul de la borne supérieure : Deux méthodes sont implémentées. La première (option 1) utilise une approche gloutonne en remplissant le sac avec les objets ayant le meilleur ratio prix/poids, en fonction de la capacité restante. La seconde (option 2) trie les objets par ratio prix/poids et calcule la borne en conséquence.

Règle de séparation : La séparation des nœuds est effectuée en fonction de l’ordre lexicographique ou selon la stratégie de recherche en profondeur. Si un nœud n’est pas sondable, de nouvelles branches sont créées.

Test d’abandon : Trois types de tests sont appliqués pour déterminer si un nœud doit être ignoré :

Faisabilité (TA),

Optimalité (TO),

Relaxation (TR).

Stratégie d’exploration : La recherche suit l’ordre lexicographique ou privilégie les branches les plus profondes en premier (DFS).

Graphe de recherche : Les structures de données trParentnodes, trChildnodes et trNamenodes sont utilisées pour représenter les relations entre les nœuds du graphe, facilitant la visualisation et la gestion du graphe de recherche.

Choix des structures de données : Les listes, comme listobjs et listvals, stockent les informations liées aux objets et leurs valeurs. Les variables importantes, telles que BestProfit et BestSol, sont passées en paramètre pour permettre leur mise à jour directe dans les fonctions. La simplicité et la lisibilité du code sont privilégiées, avec une gestion du graphe basée sur des listes pour représenter les relations parent-enfant.

- Q5. Les performances montrent que l’utilisation de la borne 1 est plus efficace que celle de la borne 2 en termes de temps d’exécution, ce qui semble cohérent avec les propriétés de chaque méthode.